In [209]:
import os
import sys
import glob
import datetime
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xray
import random
import itertools
import urllib
import json
import importlib as imp

from IPython import display

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.dates as climadates
import climatools.viz as climaviz
import climatools.misc as climamisc
import climatools.muths as climaths
import climatools.units as climaunits
import climatools.geocoords as climageocoords

#from IPython.lib.deepreload import reload as dreload
imp.reload(climadates)
imp.reload(climaviz)
imp.reload(climamisc)
imp.reload(climaths)
imp.reload(climaunits)

matplotlib.rcParams.update({'font.size': 13, 
                            'xtick.direction': 'out',
                            'ytick.direction': 'out'})

# MPACE

2 cases considered here are:

* a McICA --- RRTMG with McICA cloud overlapping, output sent to CAM
* b MaxRand --- RRTMG with CLIRAD's maximum random cloud overlapping

Both above cases use the same input state in each model nstep. 


In [210]:
iopname = 'mpace_LW_rrtmgMcICA_rrtmgMaxRand_SW_rrtmgMcICA_rrtmgMaxRand'

def case_names():
    return ['a McICA', 'b MaxRand']

# Each case here refers to a model, or a model configuration, and is assigned the path to its output data.
def get_cases():
    '''
    Returns a dictionary where the keys are cases, such
    as a name for a SCAM run, and the values are the datasets (xray)
    for the cases
    '''
    ds = xray.open_dataset('../examples/MPACE/cam5_noavg_LW_rrtmgMcICA_rrtmgMaxRand_SW_rrtmgMcICA_rrtmgMaxRand/camrun.cam.h0.2004-10-05-07171.nc',
                          decode_times = False)
    RTM1_names = [k for k in list(ds) if '_RTM1' in k]
    rrtmg_names = [name.split('_')[0] for name in RTM1_names]
    common_names = [k for k in list(ds) 
                    if k not in (RTM1_names + rrtmg_names)]
    
    ds_RTM1 = xray.Dataset()
    ds_RTM1.update(ds[common_names].copy(deep = True), inplace = True)
    ds_RTM1.update(ds[RTM1_names].copy(deep = True), inplace = True)
    ds_RTM1.rename(dict(zip(RTM1_names, rrtmg_names)), inplace = True)
    
    ds_rrtmg = xray.Dataset()
    ds_rrtmg.update(ds[common_names].copy(deep = True), inplace = True)
    ds_rrtmg.update(ds[rrtmg_names].copy(deep = True), inplace = True)     
    
       
    return dict(zip(case_names(), [ds_rrtmg, ds_RTM1]))





def compute_net_solar_fluxes(datasets):
    '''
    Compute net solar fluxes
    '''
    for casename, ds in datasets.items():
        ds['net_FS'] = ds['FDS'] - ds['FUS']
        ds['net_FS'].attrs['units'] = ds['FDS'].attrs['units']
        ds['net_FS'].attrs['long_name'] = 'net solar flux'
        ds['net_FS'].attrs['case_name'] = casename
        
        ds['net_FSC'] = ds['FDSC'] - ds['FUSC']
        ds['net_FSC'].attrs['units'] = ds['FDSC'].attrs['units']
        ds['net_FSC'].attrs['long_name'] = 'net solar flux (clear-sky)'
        ds['net_FSC'].attrs['case_name'] = casename
    return datasets




def linestyles_by_case():
    return dict(zip(case_names(), 
                    [{'colour': 'g', 'linestyle': '--'},
                     {'colour': 'b', 'linestyle': '--'}]))

In [211]:
def get_cases_difference():
    return [('b MaxRand', 'a McICA'),]

In [212]:
############### Coordinate conversion functions ################
################################################################
# *lev* is converted from hybrid levels to mbar.    
# *time* is converted from days since start to datetime.datetime objects


def convert_pressure_time_units(cases):
    '''
    Convert pressure and time units to mbar and datetime objects.
    This assumes all datasets have the same pressure and time
    coordinates.  This conversion is done early using fields such as
    hyam, which will not be needed later.
    '''
    ds = cases[random.choice(list(cases.keys()))]
    lev = climaunits.hybrid2mbar(ds, level_type = 'lev')
    ilev = climaunits.hybrid2mbar(ds, level_type = 'ilev')
    datetimes = climadates.time2datetimes(ds)
    datetimes = [climadates.\
                 UTCtime_to_localtime(datetime, 
                                      lon = climageocoords.\
                                      positivise_longitude(ds['lon'].values[0])) \
                 for datetime in datetimes]
    return ilev, lev, datetimes

In [213]:
cases = get_cases()
cases = compute_net_solar_fluxes(cases)
ilev, lev, datetimes = convert_pressure_time_units(cases)

In [214]:
################## Variables of dimensions (time, lev, lon, lat) ########################
################################################################################

In [215]:
### for pre-processing



def get_d3interests():
    '''List all interested fields here.  
    These have to be of dimensions (time, lev, lon, lat)'''
    d3interests = ['CLOUD', 'QRL', 'QRS', 'Q',
                   'CLDICE', 'CLDLIQ', 
                   'AQSNOW', 'AREI', 'AREL',
                  'AWNC', 'AWNI', 
                   'DCQ', 'DTCOND', 
                   'ICIMR', 'ICWMR', 
                   'NUMLIQ', 'NUMICE', 'T', 'RELHUM',
                  'FUL', 'FDL', 'FULC', 'FDLC', 'net_FS', 'net_FSC',
                  'FUS', 'FDS', 'FUSC', 'FDSC',
                  'TOT_CLD_VISTAU']
    return sorted(d3interests)


def gather_interests_from_cases(cases, interests):
    '''
    Creates a dictionary with keys being the cases.
    For each case is an Xray Dataset containing the interested fields
    '''
    datasets = {} 

    for case, ds in cases.items():
        datasets[case] = ds[interests].copy(deep = True)
        datasets[case].attrs['case_name'] = case
    return datasets


def convert_units(datasets, ilev, lev, datetimes):
    '''
    Convert units for certain fields for all cases in DATASETS.
    INPUT:
    ilev --- level(interface) pressure in mbar
    lev  --- layer(level) pressure in mbar
    datetimes --- time in pandas DatetimeIndex objects
    '''
    for name, ds in datasets.items():
        ds.coords['ilev'] = ('ilev', ilev, {'units': 'mbar'})
        ds.coords['lev'] = ('lev', lev, {'units': 'mbar'})
        ds.coords['time'] = ('time', datetimes, {'units': 'datetime'})
        
        for k in ['TOT_CLD_VISTAU',]:
            if k in ds:
                pass #ds[k].values[ds[k].isnull().values] = - 0.
        
        for k in ['FUS', 'FDS', 'net_FS', 'net_FSC']:
            if k in ds:
                ds[k] *= 1e3
    
        for k in ['QRL', 'QRS', 'DTCOND']:
            if k in ds:
                ds[k] *= 86400
                ds[k].attrs['units'] = 'K/day'
        
        for k in ['Q', ]:
            if k in ds:
                ds[k] *= 1e3
                ds[k].attrs['units'] = 'g/kg'
        
        for k in ['CLDICE', 'CLDLIQ', 'AQSNOW']:
            if k in ds:
                ds[k] *= 1e6
                ds[k].attrs['units'] = 'mg/kg'
                
    return datasets





def take_difference_between_cases(datasets):
    '''
    Take the difference between all cases for every field
    and return in a similar dictionary
    '''
    diff_strs = get_cases_difference()
    
    return {x + ' - ' + y: datasets[x] - datasets[y] 
            for x, y in diff_strs}


def passon_attrs_casename(datasets, diff_datasets):
    '''
    Create an attribute for each case.
    Copy over attributes to the differences.
    This is not nice, might be good to get rid of 
    the dependency on this.
    '''
    interests = get_d3interests() 
    
    for ds_name, ds in diff_datasets.items():
        ds.attrs['case_name'] = ds_name
        for interest in interests:
            ds[interest].attrs = dict(
                datasets[random.choice(list(datasets.keys()))][interest].attrs)
            ds[interest].attrs['case_name'] = ds_name
        
    for ds_name, ds in datasets.items():
        for interest in interests:
            ds[interest].attrs['case_name'] = ds_name
            
    return datasets, diff_datasets


def prep_data(cases, ilev, lev, datetimes):
    '''
    Preprocess data for plotting. 
    pressure and datetimes need to be provided for unit conversion.
    INPUT:
    ilev --- level(interface) pressure [mbar]
    lev --- layer(level) pressure [mbar]
    datetimes --- time in pandas DatetimeIndex objects    
    '''
    interests = get_d3interests()
    datasets = gather_interests_from_cases(cases, interests)
    datasets = convert_units(datasets, ilev, lev, datetimes)
    diff_datasets = take_difference_between_cases(datasets)
    datasets, diff_datasets = passon_attrs_casename(datasets, diff_datasets)
    return datasets, diff_datasets


In [216]:
def interests_using_bwr_cmap():
    return ['QRL', 'QRS',
           'DTCOND', 'DCQ']



# for writing plotted figures to disk
def writefig_contourfs(d3sets, diff_d3sets, 
                       interests, 
                       dir_label = 'example_dir_label',
                       file_label = 'example_file_label',
                       fmt = 'png'):
    '''
    For each interest, produce one figure containing contour
    plots for all cases, another figure containing contour 
    Hackish colormap selection here, basically look up a dictionary.
    '''

    for interest in interests:
        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)

        if interest in interests_using_bwr_cmap():
            cmap_name = 'bwr'
        else:
            cmap_name = 'PuBuGn'
            
        fig = climaviz.contourf_interest_for_all_cases(d3sets, interest,
                                           cmap = plt.get_cmap(cmap_name))
        fig.savefig(dirname + '/contourf_{}.{}'.format('_'.join([interest, file_label]), fmt))
        plt.close()
        
        fig = climaviz.contourf_interest_for_all_cases(diff_d3sets, interest,
                                           cmap = plt.get_cmap('bwr'))
        fig.savefig(dirname + '/contourf_{}_diff.{}'.format('_'.join([interest, file_label]), fmt))
        plt.close()
        

        
def writefig_timeaveraged_VSplots(d3sets, diff_d3sets,
                                 interests, 
                                 dir_label = 'example_dir_label', file_label = 'example_file_label',
                                 fmt = 'png'):
    for interest in interests:        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
            
        fig = climaviz.plotVS_timeaveraged_interest_for_all_cases(d3sets, diff_d3sets, 
                               interest = interest, xscale = 'linear',
                               bot_xlabels_rotate = 10., 
                               top_xlabels_rotate = 20.,
                               linestyles = linestyles_by_case())
        fig.savefig(dirname + '/plot_{}_timeaverage.{}'.format('_'.join([interest, file_label]), fmt))
        plt.close()

In [217]:
def write_d3variable_html_page(interest = 'CLOUD', 
                               dir_label = 'example_dir_label', 
                               file_label = 'example_file_label'):
    figdir = 'figs_' + dir_label
    with open(os.path.join(figdir, '_'.join([interest, file_label]) + '.html'), 
              mode = 'w', encoding = 'utf-8') as f:
        f.write('''
                <img src="contourf_{suffix}.png" height="450">
                <img src="contourf_{suffix}_diff.png" height="450">
                <br>
                <img src="plot_{suffix}_timeaverage.png" height="500">
        '''.format(figdir = figdir, suffix = '_'.join([interest, file_label])))

        
def write_d3variable_links_html(interests = ['T'],
                               dir_label = 'example_dir_label', 
                               file_label = 'example_file_label'):
    hyperlinks = ['<a href="figs_{dir_label}/{interest}_{file_label}.html">{interest}: {longname}</a>'\
                 .format(dir_label = dir_label, file_label = file_label,
                 interest = interest, 
                 longname = d3sets[random.choice(list(d3sets.keys()))][interest].attrs['long_name']) \
                 for interest in interests]
    return '<br>'.join(hyperlinks)
    

In [218]:
# prepare data and differences
d3sets, diff_d3sets = prep_data(cases, ilev, lev, datetimes) 

In [219]:
# plot and write figures to disk
interests = get_d3interests() 

writefig_contourfs(d3sets, diff_d3sets, interests, 
                   dir_label = iopname, file_label = '',
                  fmt = 'png')

writefig_timeaveraged_VSplots(d3sets, diff_d3sets, interests, 
                              dir_label = iopname, file_label = '',
                              fmt = 'png')
# generate HTML page for each interest
for interest in get_d3interests():
    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = '')


In [220]:
############### Variables of dimensions (lon, lat, time) ##########################
###################################################################################

In [221]:
def get_d2interests():
    interests = ['FSNT', 'FSNS', 'FLNT', 'FLNS',
                 'CLDLOW', 'CLDMED', 'CLDHGH', 'CLDTOT',
                 'TS', 'TREFHT', 'TROP_T', 'TROP_Z', 'TMQ', 'TGCLDIWP', 'TGCLDLWP', 
                 'PRECT', 'PRECL', 'PRECC',
                 'LHFLX', 'SHFLX',
                'SOLIN']
    return interests

In [222]:
def prep_data():

    interests = get_d2interests()

    datasets = {}
    
    for casename, ds in cases.items():
        datasets[casename] = ds[interests].copy(deep = True)
        datasets[casename].attrs['case_name'] = casename
        
    for casename, ds in datasets.items():
        ds.coords['time'] = ('time', datetimes, {'units': 'datetime',
                                                'long_name': 'time'})
        
        for k in ['PRECC', 'PRECL', 'PRECT']:
            if k in ds:
                ds[k] *= 1e3 * 86400
                ds[k].attrs['units'] = 'mm/day'
        
    return datasets

d2sets = prep_data()

In [223]:
#### Compute difference between all cases for all interested variables
def prep_differences(datasets):
    diff_strs = get_cases_difference()

    return {x + ' - ' + y: datasets[x] - datasets[y] 
            for x, y in diff_strs}    

diff_d2sets = prep_differences(d2sets)

In [224]:
#### Change/correct some of the attributes
## Some of this is needed for labelling plots easily

def prep_attributes(datasets, diff_datasets):
    
    interests = get_d2interests()
    
    for ds_name, ds in diff_datasets.items():
        ds.attrs['case_name'] = ds_name
        for interest in interests:
            ds[interest].attrs = dict(
                datasets[random.choice(list(datasets.keys()))][interest].attrs)
            ds[interest].attrs['case_name'] = ds_name
        
    for ds_name, ds in datasets.items():
        for interest in interests:
            ds[interest].attrs['case_name'] = ds_name
            
    return datasets, diff_datasets

d2sets, diff_d2sets = prep_attributes(d2sets, diff_d2sets)

In [225]:
# for writing figures to disk
def writefig_plotVS_timeseries(dsets, diff_dsets, interests,
                              sitename, fmt = 'png'):
    for interest in interests:
        fig = climaviz.plotVS_interest_for_all_cases(dsets, diff_dsets, interest = interest,
                                                    linestyles = linestyles_by_case())
        
        dirname = 'figs_{}'.format(sitename)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
            
        fig.savefig(dirname + '/plotVSts_{}.'.format(interest) + fmt)
        plt.close()

def write_d2variable_html_page(iopname = 'mpace', interest = 'FLNS'):
    figdir = 'figs_{iopname}'.format(iopname = iopname)
    with open(os.path.join(figdir, '{iopname}_{interest}.html'\
                           .format(iopname = iopname,
                                   interest = interest)),
             mode = 'w', encoding = 'utf-8') as f:
        f.write('''
        <img src="plotVSts_{interest}.png" width=700>
        '''.format(interest = interest))
        
def write_d2variables_html(iopname = 'mpace', interests = ['FLNS']):
    hyperlinks = ['<a href="figs_{iopname}/{iopname}_{interest}.html">{interest}: {longname}</a>'\
                 .format(iopname = iopname, interest = interest,
                        longname = d2sets[random.choice(list(d2sets.keys()))]\
                         [interest].attrs['long_name'])\
                 for interest in interests]
    return '<br>'.join(hyperlinks)


In [226]:
# plot and write figures to disk

writefig_plotVS_timeseries(d2sets, diff_d2sets, get_d2interests(),
                          iopname, fmt = 'png')  

# generate HTML page for each interest
for interest in get_d2interests():
    write_d2variable_html_page(iopname = iopname, interest = interest)


# Variables of dimensions (time, ilev, lon, lat)

In [227]:
display.HTML(write_d3variable_links_html(interests = get_d3interests(), dir_label = iopname, file_label = ''))

# Variables of dimensions (lon, lat, time)

In [228]:
display.HTML(write_d2variables_html(iopname = iopname, interests = get_d2interests()))

In [229]:
def slice_in_time(cases, time = slice(None)):
    return {casename: ds.sel(time = time) for casename, ds in cases.items()}



In [233]:
# plot and write figures to disk
#interests = get_d3interests()

#writefig_contourfs(d3sets_subtime, diff_d3sets_subtime, interests, 
#                   dir_label = iopname, file_label = which_timeslice,
#                  fmt = 'png')

#writefig_timeaveraged_VSplots(d3sets_subtime, diff_d3sets_subtime, interests, 
#                              dir_label = iopname, file_label = which_timeslice,
#                              fmt = 'png')
#pass

In [234]:
# generate HTML page for each interest
#for interest in get_d3interests():
#    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = which_timeslice)


#display.HTML((write_d3variable_links_html(interests = interests, dir_label = iopname, file_label = which_timeslice)))

In [122]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')